# The Benchmarking Interlude

In [25]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
os.getcwd()
os.chdir('/Users/fizz/Document/Notes/Python/codes')

'/Users/fizz/Document/Notes/Python/codes'

In [40]:
print(open('timer3.py').read())

"""
Same usage as timer2.py, but uses 3.X keyword-only default arguments instead of dict pops for simpler code. No need to hoist range() out of tests in 3.X: always a generator in 3.X, and this can't run on 2.X. 
"""
import time, sys
timer = time.clock if sys.platform[:3] == 'win' else time.time
def total(func, *pargs, _reps=1000, **kargs):
    start = timer()
    for i in range(_reps):
        ret = func(*pargs, **kargs)
    elapsed = timer() - start
    return (elapsed, ret)
def bestof(func, *pargs, _reps=5, **kargs):
    best = 2 ** 32
    for i in range(_reps): start = timer()
        ret = func(*pargs, **kargs)
    elapsed = timer() - start
    if elapsed < best: best = elapsed
    return (best, ret)
def bestoftotal(func, *pargs, _reps1=5, **kargs):
    return min(total(func, *pargs, **kargs) for i in range(_reps1))



In [31]:
import timer
timer.total(1000, pow, 2, 1000)[0]
timer.bestof(1000, str.upper, 'spam')
timer.bestof(1000, pow, 2, 1000000)[0]

timer.bestof(50, timer.total, 1000, str.upper, 'spam')
timer.bestoftotal(50, 1000, str.upper, 'spam')

min(timer.total(1000, str.upper, 'spam') for i in range(50))

0.0009908676147460938

(0.0, 'SPAM')

0.002838611602783203

(6.985664367675781e-05, (5.888938903808594e-05, 'SPAM'))

(7.081031799316406e-05, (5.91278076171875e-05, 'SPAM'))

(5.817413330078125e-05, 'SPAM')

## Timing Iterations and Pythons with timeit

In [54]:
import timeit
timeit.repeat(stmt='[x ** 2 for x in range(1000)]', number=1000, repeat=5)

min(timeit.repeat(number=1000, repeat=3,
      stmt = 'L = [1, 2, 3, 4, 5]\nfor i in range(len(L)): L[i] += 1'))
min(timeit.repeat(number=1000, repeat=3,
      stmt = 'L = [1, 2, 3, 4, 5]\ni=0\nwhile i < len(L):\n\tL[i] += 1\n\ti += 1'))

[0.23632496899881517,
 0.21857701300177723,
 0.2199981840021792,
 0.2142235770006664,
 0.2227727040008176]

0.0005589329994108994

0.0006874760001664981

The **timeit** module also allows you to provide **setup** code that is run in the main statement's scope, but whose time is not charged to the main statement's total - potentially useful for initialization code you wish to exclude from total time, such as imports of required modules, test function definition, and test data creation. Because they're run in the same scope. any names created by setup code are available to the main test statement; names defined in the interactive shell generally are not.

In [58]:
timeit.timeit(stmt='[x ** 2 for x in range(1000)]', number=1000)   # Total time

timeit.Timer(stmt='[x ** 2 for x in range(1000)]').timeit(1000)      # Class API

def testcase():
    y = [x ** 2 for x in range(1000)]     # Callable objects or code strings
min(timeit.repeat(stmt=testcase, number=1000, repeat=3))

0.2342106709984364

0.22296444600215182

0.2093809809994127

In [63]:
X = 99
def selector():
    import __main__        # Import enclosing module
    print(__main__.X)       # Qualify to get global version of name
    X = 88
    print(X)
selector()

99
88


As noted briefly before, mutable values for default arguments can retain state between calls, though this is often unexpected. In general, defaut argument values are evaluated and saved once when a **def** statement is run, not each time the resulting function is later called. Internally, Python saves one object per default argument attached to the function itself.

In [70]:
def saver(x=[]):
    x.append(1)
    print(x)
saver([2])    # Default not used
saver()
saver()
saver([2])
saver()

[2, 1]
[2, 2, 1]
[1]
[1, 1]
[2, 1]
[1, 1, 1]
